In [1]:
import pickle 
import pandas as pd 
from datetime import datetime
df = pd.read_hdf('../../data/processed/train/aggr_train_zeros_25feat.hdf')
df_train_pca = pd.read_hdf('../../data/processed/train/pca.hdf')
threshold = datetime(2015, 9, 1)

df['date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str) + '-1')
pca_cols = ['pca_{}'.format(i) for i in range(4)]
df[pca_cols]= df_train_pca[pca_cols]

In [2]:
from sklearn.metrics import r2_score, mean_squared_error
import xgboost as xgb
from math import sqrt
def get_preds_trained_on_all(model, df_X):
    return model.predict(df_X).clip(0, 20)


def previous_value_preds(df_X):
    df_X = df_X.reindex(columns=df_X_train.columns, fill_value=0)
    return df_X['count_aggr_1_month_ago_shop_id_item_id'].clip(0, 20)


def validate(model, df_y_valid, df_X_valid):
    df_y_valid = df_y_valid.copy().clip(0, 20)
    df_y_valid_preds = get_preds_trained_on_all(model, df_X_valid)
    # df_y_valid_preds = previous_value_preds(df_X_valid)
    print('r2 score: ', r2_score(df_y_valid, df_y_valid_preds))
    print('RMSE: ', sqrt(mean_squared_error(df_y_valid, df_y_valid_preds)))

In [7]:
from sklearn.neighbors import KNeighborsRegressor
df_train = df[df['date'] < threshold]
#df_train = df
df_X_train, df_y_train = df_train.drop(['date', 'item_cnt_day'], axis=1), df_train['item_cnt_day'].clip(0, 20)
model = KNeighborsRegressor(n_neighbors=3, n_jobs=-1)
model.fit(df_X_train, df_y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=-1, n_neighbors=3, p=2,
          weights='uniform')

In [9]:
df_valid = df[df['date'] >= threshold]

df_X_valid, df_y_valid = df_valid.drop(['date', 'item_cnt_day'], axis=1), df_valid['item_cnt_day']
validate(model, df_y_valid, df_X_valid)

r2 score:  -5.727450836645142
RMSE:  2.8606795337747952


In [15]:
df_valid = df[df['date'] >= threshold]

df_X_valid, df_y_valid = df_valid.drop(['date', 'item_cnt_day'], axis=1), df_valid['item_cnt_day']
validate(model, df_y_valid, df_X_valid)

r2 score:  -7.499413355042682
RMSE:  3.215425251370679


In [10]:
df_valid = df[df['date'] >= threshold]
df_X_valid['preds'] = get_preds_trained_on_all(model, df_X_valid)
a = df_X_valid['preds']
a.to_hdf('../../data/processed/model_valid/pre_valid_train_clip/KNN_3.hdf', 'KNN_3')

In [26]:
df_test = pd.read_hdf('../../data/processed/test/aggr_test_25feat.hdf')
df_test_pca = pd.read_hdf('../../data/processed/test/pca.hdf')
df_test[pca_cols] = df_test_pca[pca_cols]
df_X_test = df_test.drop(['ID'], axis=1)
df_test['KNN_3_preds'] = get_preds_trained_on_all(model, df_X_test)
df_test[['ID', 'KNN_3_preds']].to_hdf('../../data/processed/model_valid/KNN_3_test.hdf', 'KNN3_test')

In [4]:
df_test = pd.read_hdf('../../data/processed/test/aggr_test_25feat.hdf')
df_test_pca = pd.read_hdf('../../data/processed/test/pca.hdf')
df_test[pca_cols] = df_test_pca[pca_cols]
df_X_test = df_test.drop(['ID'], axis=1)
df_test['KNN_3_preds'] = get_preds_trained_on_all(model, df_X_test)
df_test[['ID', 'KNN_3_preds']].to_hdf('../../data/processed/model_valid/all_train/KNN_3_test.hdf', 'KNN3_test')

In [6]:
df_test = pd.read_hdf('../../data/processed/test/aggr_test_25feat.hdf')
df_test_pca = pd.read_hdf('../../data/processed/test/pca.hdf')
df_test[pca_cols] = df_test_pca[pca_cols]
df_X_test = df_test.drop(['ID'], axis=1)
df_test['KNN_3_preds'] = get_preds_trained_on_all(model, df_X_test)
df_test[['ID', 'KNN_3_preds']].to_hdf('../../data/processed/model_valid/all_train_clip/KNN_3_test.hdf', 'KNN3_test')

In [23]:
df_X_valid.columns

Index(['count_aggr_1_month_shop_id_item_id', 'count_aggr_year_shop_id_item_id',
       'count_aggr_6_months_shop_id_item_id',
       'count_aggr_3_months_shop_id_item_id', 'month', 'day_of_year',
       'count_aggr_year_mall_item_id', 'count_aggr_year_city_item_category_id',
       'count_aggr_1_month_shop_id', 'count_last_week_shop_id_item_id',
       'expected_sales', 'count_last_week_city_item_id',
       'count_last_week_item_id', 'count_aggr_1_month_mall',
       'count_aggr_year_city', 'count_aggr_1_month_ago_last_big_category',
       'count_aggr_year_city_item_id', 'count_aggr_6_months_mall_item_id',
       'count_aggr_year_shop_id', 'count_aggr_6_months_first_big_category',
       'count_aggr_2_quartals_ago_shop_id_item_id',
       'count_last_week_item_category_id',
       'count_last_week_city_item_category_id',
       'count_aggr_6_months_mall_item_category_id',
       'count_aggr_1_month_ago_mall_item_category_id', 'year', 'pca_0',
       'pca_1', 'pca_2', 'pca_3', 'preds'

In [22]:
df_X_test.shape

(214200, 26)